# Paper Rock Sessiors
This notebook is using Python Tensorflow/Keras to build a Convolution Neural Network (CNN) to classify an image of human hand into "Paper", "Rock", or "Scissors".

How to use the notebook:
1.   Upload the dataset of images to the notebook Files.
2.   Make sure the machine is running with GPU. 
Go to *Runtime > Change Runtime Type >  Hardware Acceleration > GPU*
3.   Run each code block in sequence

Happy coding 🦾

In [ ]:
!unzip rps.zip -d /content/rps

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

rock_dir = "/content/rps/train/rock"
paper_dir = "/content/rps/train/paper"
scissors_dir = "/content/rps/train/scissors"

# Printing the number of images in each folder (for double checking)
print(len(os.listdir(rock_dir)))
print(len(os.listdir(paper_dir)))
print(len(os.listdir(scissors_dir)))

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Setting a data augmetation generator for training
train_dir = "/content/rps/train"
train_datagen = ImageDataGenerator(rescale=1./255, \
                                   rotation_range=90, \
                                   width_shift_range=0.2, \
                                   height_shift_range=0.2, \
                                   shear_range=0.1, \
                                   zoom_range=0.2, \
                                   horizontal_flip=True, \
                                   fill_mode='nearest', \
                                   brightness_range=(0.3,0.7)
                                   )
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150,150), \
                                                    batch_size=20, class_mode="categorical", color_mode='grayscale')

# Setting a data generator for validation data
valid_dir = "/content/rps/valid"
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(150,150), \
                                                    batch_size=20, class_mode="categorical", color_mode='grayscale')

# Setting a data generator for testing data
test_dir = "/content/rps/test"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150,150), \
                                                    batch_size=20, class_mode="categorical", color_mode='grayscale')

In [ ]:
# plotting sample of the training images
# Note: you can save the image by right-clicking on it
import matplotlib.pyplot as plt
x_batch, y_batch = next(train_generator)
f, axarr = plt.subplots(4,4, figsize=(12, 12))
for i in range(4):
  for j in range(4):
    image = x_batch[i*4+j]
    axarr[i,j].imshow(image,cmap='gray')
plt.show()

In [ ]:
# This code will create the model
# You can add more layers, change the number of filters (kernals), 
# and improve the accuracy!

from tensorflow.keras import layers

def create_model():
  model = keras.Sequential()

  # Layer 1: Convolution with 64 filters (3x3) followed by Max Polling layer
  model.add(layers.Conv2D(64, (3,3), input_shape=(150,150,1), activation='relu'))
  model.add(layers.MaxPooling2D())

  # Layer 2: Convolution with 64 filters (3x3) followed by Max Polling layer
  model.add(layers.Conv2D(64, (3,3), activation='relu'))
  model.add(layers.MaxPooling2D())

  # Layer 3: Convolution with 128 filters (3x3) followed by Max Polling layer
  model.add(layers.Conv2D(128, (3,3), activation='relu'))
  model.add(layers.MaxPooling2D())

  # Layer 4: Convolution with 128 filters (3x3) followed by Max Polling layer
  model.add(layers.Conv2D(128, (3,3), activation='relu'))
  model.add(layers.MaxPooling2D())

  # Layer 5: Fully connected Layer
  model.add(layers.Flatten())
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(512, activation='relu'))

  # Layer 6: Output Layer
  model.add(layers.Dense(3, activation='softmax'))
  return model

model = create_model()

# Print a summary about the model (for double checking)
model.summary()

In [ ]:
# Start training for 15 epochs (you can choose less or more epochs)
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics=['acc'])
hist = model.fit(train_generator, epochs=15, validation_data=valid_generator)

In [ ]:
# Plotting the training accuracy VS validation accuracy
# The model performs well if both are increasing together
# The model performs bad if the validation accuracy drops
# Note: you can save the image by right-clicking on it

import matplotlib.pyplot as plt

acc = hist.history['acc']
val_acc = hist.history['val_acc']
epochs = range(len(acc))

plt.plot(epochs, acc, label="training accuracy")
plt.plot(epochs, val_acc, label = "validation accuracy")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Evaluate the model on the test data
print("Evaluate on test data")
results = model.evaluate(test_generator)
print("test loss, test acc:", results)

In [ ]:
!mkdir -p saved_model
model.save('models/rps_model')

In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# define the directory for .pb model
pb_model_path = "/content"
# define the name of .pb model
pb_model_name = "rps_model.pb"
# create directory for further converted model
os.makedirs(pb_model_path, exist_ok=True)
# get model TF graph
tf_model_graph = tf.function(lambda x: model(x))
# get concrete function
tf_model_graph = tf_model_graph.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
# obtain frozen concrete function
frozen_tf_func = convert_variables_to_constants_v2(tf_model_graph)
# get frozen graph
frozen_tf_func.graph.as_graph_def()
# save full tf model
tf.io.write_graph(graph_or_graph_def=frozen_tf_func.graph,
                  logdir=pb_model_path,
                  name=pb_model_name,
                  as_text=False)

In [ ]:
# testing
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical


model = tf.keras.models.load_model('rps.h5')
#model.summary()

img = cv2.imread("/content/rps/valid/rock/testrock01-00_png.rf.9fb3b294595cff53ba76bef13e210814.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = img/255.0

img = cv2.resize(img, (150,150))
plt.imshow(img, cmap='gray')
img = np.expand_dims(img, axis=0) # image shape is (1,150,150,1)
#print(img.shape)
target = np.array([[0,1,0]])
#print(target.shape)

output = model.predict(img, verbose=False)
imagenet_class_id = np.argmax(output)
imagenet_labels = ['Paper', "Rock", "Scissor"]
print(imagenet_labels[imagenet_class_id])
loss, acc = model.evaluate(img, target, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))